# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [2]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [3]:
### General parameters
max_steps = 1e7 # Set maximum number of steps to run environment.
run_path = "Ball3" # The sub-directory name for model and summary statistics
load_model = True # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 1000 # Frequency at which to save training statistics.
save_freq = 10000 # Frequency at which to save model.
env_name = "Ball2" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-4 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 1e-4 # Model learning rate.
hidden_units = 64 # Number of units in hidden layer.
batch_size = 64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [4]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file, worker_id=5)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'Ball3DAcademy' started successfully!


Unity Academy name: Ball3DAcademy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: Ball3DBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 14
        Action space type: continuous
        Action space size (per agent): 2
        Memory space size (per agent): 0
        Action descriptions: , 


### Train the Agent(s)

In [7]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
config = tf.ConfigProto(device_count = {'GPU': 0})

with tf.Session(config = config) as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
            #export_graph(model_path, env_name+"_"+str(steps))
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/Ball3\model-160000.cptk


INFO:tensorflow:Restoring parameters from ./models/Ball3\model-160000.cptk


Saved Model
Step: 161000. Mean Reward: 40.15652173913009. Std of Reward: 41.87743842179486.
Step: 162000. Mean Reward: 34.91111111111081. Std of Reward: 40.96739745603746.
Step: 163000. Mean Reward: 37.40689655172377. Std of Reward: 43.03975652919864.
Step: 164000. Mean Reward: 21.18793103448259. Std of Reward: 35.214828761260115.
Step: 165000. Mean Reward: 40.28666666666617. Std of Reward: 46.24907014080204.
Step: 166000. Mean Reward: 40.693333333332845. Std of Reward: 45.83553885602486.
Step: 167000. Mean Reward: 38.75666666666619. Std of Reward: 45.54256385209515.
Step: 168000. Mean Reward: 42.35384615384571. Std of Reward: 43.85609536417558.
Step: 169000. Mean Reward: 26.92040816326504. Std of Reward: 39.229065544654915.
Step: 170000. Mean Reward: 15.901470588235307. Std of Reward: 23.61093024955037.
Saved Model
Step: 171000. Mean Reward: 62.9058823529405. Std of Reward: 40.850120175331796.
Step: 172000. Mean Reward: 43.3655172413788. Std of Reward: 46.05010221907845.
Step: 173000.

Saved Model
Step: 261000. Mean Reward: 11.936363636363621. Std of Reward: 20.035632170556152.
Step: 262000. Mean Reward: 22.484482758620587. Std of Reward: 32.60539833516084.
Step: 263000. Mean Reward: 23.202173913043364. Std of Reward: 34.30360919742604.
Step: 264000. Mean Reward: 25.01730769230752. Std of Reward: 36.77278321067283.
Step: 265000. Mean Reward: 24.697560975609615. Std of Reward: 36.060352444774814.
Step: 266000. Mean Reward: 16.655813953488273. Std of Reward: 30.87792394459443.
Step: 267000. Mean Reward: 4.100881057268728. Std of Reward: 10.40499706782501.
Step: 268000. Mean Reward: 2.995238095238097. Std of Reward: 8.438326963305439.
Step: 269000. Mean Reward: 3.6019157088122564. Std of Reward: 9.302858507291564.
Step: 270000. Mean Reward: 3.4190311418685257. Std of Reward: 9.437106568021473.
Saved Model
Step: 271000. Mean Reward: 1.0440000000000003. Std of Reward: 1.8040872114913593.
Step: 272000. Mean Reward: 1.332112068965519. Std of Reward: 3.419988004797375.
Step:

Saved Model
Step: 361000. Mean Reward: 3.547407407407416. Std of Reward: 8.00758741703313.
Step: 362000. Mean Reward: 3.3972972972973015. Std of Reward: 5.934471026414412.
Step: 363000. Mean Reward: 2.3773913043478268. Std of Reward: 3.4494459341578456.
Step: 364000. Mean Reward: 2.573559322033899. Std of Reward: 4.228416285133162.
Step: 365000. Mean Reward: 4.3981566820276505. Std of Reward: 9.517037659837651.
Step: 366000. Mean Reward: 7.6301587301587634. Std of Reward: 13.156387004889538.
Step: 367000. Mean Reward: 5.635359116022091. Std of Reward: 14.650569672264012.
Step: 368000. Mean Reward: 7.072932330827062. Std of Reward: 15.048019017925238.
Step: 369000. Mean Reward: 8.077586206896546. Std of Reward: 18.505227994788857.
Step: 370000. Mean Reward: 19.448148148148043. Std of Reward: 31.304151545410605.
Saved Model
Step: 371000. Mean Reward: 11.924778761061916. Std of Reward: 23.96654944308175.
Step: 372000. Mean Reward: 9.703960396039607. Std of Reward: 19.0482083603945.
Step: 

KeyboardInterrupt: 

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [6]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/Ball3\model-160000.cptk


INFO:tensorflow:Restoring parameters from ./models/Ball3\model-160000.cptk


INFO:tensorflow:Froze 7 variables.


INFO:tensorflow:Froze 7 variables.


Converted 7 variables to const ops.
